딥러닝의 발전으로 인해 CV분야에서는 CNN을 활용한 모델이 화두가 되고 있으며, 실제로 그 성능 또한 나날이 발전하고 있는 추세입니다. 그리고 그에 따라 모델을 구성하는 신경망의 깊이가 깊어지며, 깊은 신경망에 대해서도 학습을 잘 하고 있습니다. 하지만, 어느 정도의 깊이에 도달하면, 신경망이 깊어질수록 gradient vanishing/exploding 문제로 인해 학습이 잘 이루어지지 않습니다. 그리고 이러한 문제를 해결하기 위해서 정규화 레이어의 추가, 가중치 초기화 등을 통해 꽤 깊은 모델도 학습이 가능하였지만, 여전히 모델이 깊어지다 보면 어느 순간 더 얕은 모델의 성능보다 더 나빠지는 현상이 발생합니다. 이러한 degradation 문제를 해결하기 위해서 resenet이 제안되었습니다. 즉, 깊이가 깊은 상태에서 학습을 이미 많이 진행한 경우에 대해서는 weight들의 분포가 균등하지 않고, backpropagation시 기울기가 충분하지 않아 학습을 안정적으로 할 수 없는 문제가 발생합니다. 이러한 문제는 over-fitting으로 착각할 수 있지만 실제로는 아닙니다. 

<img src='./imgs/resnet0.png' width=600>

### Skip/shortcut connection

기존의 뉴럴넷의 학습 목적이 입력($x$)을 타겟값($y$)으로 맵핑하는 함수  $H(x)$를 찾는 것이라고 한다면, 뉴럴넷은  $H(x) − y$ 를 최소화 하는 방향으로 학습을 진행한다. 이 때  $x$와 짝지어진  $y$는 사실  $x$를 대변하는 것으로, 특히 이미지 분류 문제에서는 네트워크의 입/출력을 의미상 같게끔 맵핑해야한다. 그래서 ResNet에서는 관점을 바꿔 네트워크가  $H(x) − x$를 얻는 것으로 목표를 수정하였다. 입력과 출력의 처이를  $F(x) = H(x) − x$라고 정의를 하고 네트워크는 이 $F(x)$를 찾는 것이다. 

이 $F(x)$는 잔차라고 할 수 있고, 이렇게 잔차를 학습하는 것을 Residual learning, Residual mapping이라고 한다. 결과적으로 출력  $H(x) = F(x) + x$가 된다. 이렇게 네트워크의 입력과 출력이 더해진 것을 다음 레이러의 입력으로 사용하는 것을 스킵연결(skip connection) 이라고 한다. 기존의 뉴럴넷은 $H(x)$가 어떻게든 정답과 같게 만드는 것이 목적이었다면, 이제 입력과 출력 사이의 잔차를 학습하는 것, 즉 최적의 경우 $F(x) = 0$이 되어야하므로 학습의 목표가 이미 정해져 있기 때문에 학습 속도가 빨라질 것이고, 네트워크가 잔차를 학습하고 나면, 입력값의 작은 변화에도 민감하게 반응 할 것이다라는 것이 ResNet의 가설이다.

<img src='./imgs/resnet.png'>

스킵 연결을 구현 하는 것은 덧셈 연산의 추가 만으로 가능하다. 이는 추가적인 연산량이나 파라미터가 많이 필요하지 않다. 또한 back propagation을 할 때, identiy mapping($x$)를 미분하면 적어도 1이상의 값이 나오기 때문에 최소한의 기울기를 만들어 학습이 되지 않던 현상을 최소화하였습니다. $H(x)$와 $F(x) + x$가 천천히 근사하는 방식은 같더라도 학습의 속도나 안정성으로 볼때, skip connection이 훨씬 더 효율적입니다. 

이 때, $F(x)$와 $x$의 dimension은 동일해야하므로 linear projection ($W_s$)를 적용할 수도 있습니다. 
$$ y = F(x, W_i) + x  or y = F(x, W_i) + W_sx$$
$$ F = W_{i+1}*RELU(W_ix) $$

### ResNet의 구조

ResNet의 구조는 Residual Block과 Identity Block으로 이루어져있다. 이 또한 같은 연산이지만, Identity Block에서는 입력값과 출력값의 크기를 맞추어주는 작업이 필요할 뿐이다. 다음 그림은 Residual Block(left)과 Identity Block(right) 그리고 ResNet50의 구조를 간략히 표현 한 것이다. ResNet50이란 컨볼루션(convolution) 연산과 fully connected layer만 계산 했을 때, 총 레이어 갯수가 50개가되는 ResNet 모델을 뜻한다.

<img src='./imgs/resnet1.png'>

<img src='./imgs/resnet2.png'>


ResNet에서는 첫번째 레이어(7x7 컨볼루션)를 제외하고는 모든 컨볼루션 연산에 3x3 이하 크기의 커널이 사용되었고, 피쳐맵의 크기가 같은 레이어는 출력 피쳐맵 갯수가 동일하다. 그리고 피쳐맵의 크기가 반으로 작아지는 경우 출력 피쳐맵의 갯수가 2배가 된다. pooling은 거의 사용되지 않고 컨볼루션 연산의 스트라이드(stride)를 2로 하여 피쳐맵의 크기를 줄였다. 이미지가 반으로 작아진 경우, Identity Block이 사용되며, 입력값을 바로 더하지 않고, 1x1 컨볼루션 연산을 스트라이드 2로 설정하여 피쳐맵의 크기와 갯수를 맞추어준 다음 더해준다. 이를 프로젝션 숏컷(projection shortcut)이라고도 한다.

#### bottleneck layer
ResNet50 부터는 연산량의 줄이기 위해 Residual Block 내에, 1x1, 3x3, 1x1 컨볼루션 연산을 쌓았다. Inception에서 배웠던 것과 같은 개념이다. 1x1 컨볼루션 연산으로 피쳐맵의 갯수를 줄였다가 3x3을 거친 후, 1x1 컨볼루션 연산으로 차원을 늘려준다. 이 과정이 병목 같다 하여 병목레이어(bottleneck layer)라고 부른다.

#### Pre-activation Residual Unit

ResNet 저자들은 후속 논문에서 더 개선된 skip connection 방법을 제시했다. 기존의 skip connection은 출력과 입력이 더해진 후 활성화 함수(ReLu)를 통과하였다. 활성화 함수를  $f(⋅)$ 이라고 하고, 수식으로 표현하면  $H(x) = f(F(x) + x)$가 된다. 뒤에 따라오는 비선형 활성화 함수 때문에 다음 레이어의 입력 값에 이전 레이어의 입력이 그대로 반영되지는 않는다. 하나의 Residual Block 만 생각하면 큰 차이가 없겠지만, 이를 여러 개를 연결 하면 차이가 발생할 수도 있다. 그래서,  $F(x)$안에 활성화함수를 반영한 다음  $F(x)$와 입력값을 더해준다. 수식으로 표현 하면,  $H(x) = F(x) + x$ 이다. 이를 그림으로 나타내면 다음과 같다.

<img src='./imgs/resnet3.png'>

이렇게 바꾸면, $l + 1$번째의 레이어의 입력값  $x_{l + 1} = x_l + F(x_l)$로 나타낼 수 있어 수식이 기존 보다 더 간단해 진다. 이를 더 연결해 보면, 다음과 같다.

$$x_{l + 1} = x_l + F(x_l) = x_{l − 1} + F(x_{l − 1}) + F(x_l) ⋯ = x_0 + \sum_i^lF(x_i)$$
 
각 레이어간의 관계가 더하기로 표현 되기 때문에 수학적으로 좀 더 유용하다. 그리고 실제로 논문에서의 실험결과를 확인하면 개선된 구조를 사용했을 때, 더 나은 결과를 가져온다. Keras에서 제공하는 ResNet50에는 이 구조가 적용되어 있다.